**Helper notebook**

In [ ]:
%run nb_helper

**Define a logging dataframe**

In [ ]:
dfLogging = pd.DataFrame(columns = ['LoadId','NotebookId', 'NotebookName', 'WorkspaceId', 'SourceWorkspaceName','TargetWorkspaceName','Item', 'CellId', 'Timestamp', 'ElapsedTime', 'Message', 'ErrorMessage'])
vContext = mssparkutils.runtime.context
vNotebookId = vContext["currentNotebookId"]
vLogNotebookName = vContext["currentNotebookName"]
vWorkspaceId = vContext["currentWorkspaceId"] # where the notebook is running, to not confuse with source and target workspaces

**Parameters --> convert to code for debugging the notebook. otherwise, keep commented as parameters are passed from DevOps pipelines**

pSourceWorkspaceId = ""
pTargetWorkspaceId = ""
pDebugMode = "yes"

**Resolve the source and target workspaces**

In [ ]:
vSourceWorkspaceName = fabric.resolve_workspace_name(pSourceWorkspaceId)
vTargetWorkspaceName = fabric.resolve_workspace_name(pTargetWorkspaceId)
vSourceWorkspaceId = pSourceWorkspaceId
vTargetWorkspaceId = pTargetWorkspaceId

**List of notebooks in source workspace --> semantic link labs have no function as of 22.02.2025**

In [ ]:
df_source_items = fabric.list_items(workspace=vSourceWorkspaceName)
df_source_notebooks = df_source_items[df_source_items['Type']=='Notebook']

**Verify that there is a least one notebook in the source workspace**

In [ ]:
if df_source_notebooks.empty:
    vMessage = f"workspace <vSourceWorkspaceName> have 0 notebook. post-update is not required."

    # Display an exit message
    display(Markdown("### ✅ Notebook execution stopped successfully!"))

    # Exit without error
    mssparkutils.notebook.exit(vMessage)

**Update notebook dependencies**

In [ ]:
# get the list of data pipelines in the target workspace
df_notebooks = notebookutils.notebook.list(workspaceId=vTargetWorkspaceId)
# df_notebooks

for notebook in df_notebooks:

    # get the notebook id and display name
    vNotebookId = notebook.id
    vNotebookName = notebook.displayName
   

    # get the current notebook definition
    vNotebookDefinition = notebookutils.notebook.getDefinition(name=vNotebookName, workspaceId=vSourceWorkspaceId) 
    vNotebookJson = json.loads(vNotebookDefinition)

    # update lakehouse dependencies
    try:

        # check and remove any attached lakehouses
        if 'dependencies' in vNotebookJson['metadata'] \
            and 'lakehouse' in vNotebookJson['metadata']['dependencies'] \
            and vNotebookJson['metadata']["dependencies"]["lakehouse"] is not None:

            vCurrentLakehouse = vNotebookJson['metadata']['dependencies']['lakehouse']

            if 'default_lakehouse_name' in vCurrentLakehouse:

                vNotebookJson['metadata']['dependencies']['lakehouse'] = {}
                print(f"attempting to update notebook <{vNotebookName}> with new default lakehouse: {vCurrentLakehouse['default_lakehouse_name']} in workspace <{vTargetWorkspaceName}>.")

                # update new notebook definition after removing existing lakehouses and with new default lakehouseId
                notebookutils.notebook.updateDefinition(
                    name = vNotebookName,
                    content  = json.dumps(vNotebookJson),  
                    defaultLakehouse = vCurrentLakehouse['default_lakehouse_name'],
                    defaultLakehouseWorkspace = vTargetWorkspaceId,
                    workspaceId = vTargetWorkspaceId
                )

                print(f"updated notebook <{vNotebookName}> in workspace <{vTargetWorkspaceName}>.")

            else:
                print(f'no default lakehouse set for notebook <{vNotebookName}>, ignoring.')

        vMessage = f"succeeded"
        dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, vSourceWorkspaceName, vTargetWorkspaceName, vNotebookName, 'update lakehouse dependencies', datetime.now(), None, vMessage, ''] 
    except Exception as e:
        vMessage = f"failed"
        dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, vSourceWorkspaceName, vTargetWorkspaceName, vNotebookName, 'update lakehouse dependencies', datetime.now(), None, vMessage, str(e) ] 
        if pDebugMode == "yes":
            print(str(e))

    # update warehouse dependencies
    try:
        if 'dependencies' in vNotebookJson['metadata'] and 'warehouse' in vNotebookJson['metadata']['dependencies']:
            
            #fetch existing details
            vCurrentWarehouse = vNotebookJson['metadata']['dependencies']['warehouse']
            vCurrentWarehouseId = vCurrentWarehouse['default_warehouse']
            vCurrentWarehouseName =  fabric.resolve_item_name(item_id = vCurrentWarehouseId, workspace=vSourceWorkspaceId)
            vTargetWarehouseId = fabric.resolve_item_id(item_name = vCurrentWarehouseName, type='Warehouse', workspace=vTargetWorkspaceId)

            if 'default_warehouse' in vCurrentWarehouse:

                print(f"attempting to update notebook {vNotebookName} with new default warehouse: {vTargetWarehouseId} in {vTargetWorkspaceName}")
            
                # update new notebook definition after removing existing lakehouses and with new default lakehouseId
                vNotebookJson['metadata']['dependencies']['warehouse']['default_warehouse'] = vTargetWarehouseId
                for warehouse in vNotebookJson['metadata']['dependencies']['warehouse']['known_warehouses']:
                    if warehouse['id'] == vCurrentWarehouseId:
                        warehouse['id'] = vTargetWarehouseId
                # print(json.dumps(vNotebookJson, indent=4))
                notebookutils.notebook.updateDefinition(
                    name = vNotebookName,
                    content  = json.dumps(vNotebookJson),
                    workspaceId = vTargetWorkspaceId
                )
                print(f"updated notebook {vNotebookName} in {vTargetWorkspaceName}")

            else:
                print(f"no default warehouse was found in the source notebook {vNotebookName} there cannot set default for target")

        vMessage = f"succeeded"
        dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, vSourceWorkspaceName, vTargetWorkspaceName, vNotebookName, 'update warehouse dependencies', datetime.now(), None, vMessage, ''] 
    except Exception as e:
        vMessage = f"failed"
        dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, vSourceWorkspaceName, vTargetWorkspaceName, vNotebookName, 'update warehouse dependencies', datetime.now(), None, vMessage, str(e) ] 
        if pDebugMode == "yes":
            print(str(e))

**Logging**

In [ ]:
try:
    # perform the conversion of columns
    dfLogging = dfLogging.astype({
            "LoadId": "string",	
            "NotebookId": "string", 	
            "NotebookName": "string", 
            "WorkspaceId": "string", 
            "SourceWorkspaceName" : "string",
            "TargetWorkspaceName" : "string",
            "Item":"string",
            "CellId": "string", 
            "Timestamp": "datetime64[ns]", 
            "ElapsedTime": "string", 
            "Message": "string", 
            "ErrorMessage" : "string"
        })

    # save panda dataframe to a spark dataframe 
    sparkDF_Logging = spark.createDataFrame(dfLogging) 

    # save to the lakehouse
    sparkDF_Logging.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable("staging.notebook_logging_cicd")

except Exception as e:
    vMessage = "saving logs to the lakehouse failed"
    if pDebugMode == "yes":
        print(str(e))